# 項目26-28
担当：須藤

In [32]:
# 必要なモジュール
import logging

# 項目26 functools.wrapsを使って関数デコレータを定義する

Python には、関数に適用できるデコレータの特別な構文がある。

デコレータは、ラップする関数への呼び出しの前後で追加コードを実行することができる。

これによって、入力の引数や戻り値にアクセスして値を変更したり、例外を送出したりできる。

この機能は、セマンティクス強化、デバッグ、関数登録などを行うのに役立つ。

例えば、関数呼び出しの引数と戻り値を出力したいとする。

これは、再帰関数で関数呼び出しの入れ子になったスタックをデバッグするときに、特に有用である。

そのようなデコレータを `*args` と `**kwargs` を使い、全パラメータをラップした関数に渡すことで次のように定義する(「項目22 可変長位置引数を使って、見た目をすっきりさせる」 および 「項目23 キーワード引数にオプションの振る舞いを与える」参照)。

In [73]:
# Example 1
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

このデコレータを記号を用いて関数に適用する。

In [74]:
# Example 2
@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

`@` 記号は、デコレータがラップする関数を引数として呼び出して、 戻り値を同じスコープの元々の
名前に代入することと等価である。

In [60]:
# Example 3
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

fibonacci = trace(fibonacci)

このデコレータ付きの関数を呼び出すと、`fibonacci` を実行する前後でラッパーのコードが実行されます。 再帰スタックのレベルごとに引数と戻り値を出力します。

In [75]:
# Example 4
fibonacci(4)

fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci((4,), {}) -> 3


3

これはきちんと動作しているようにも見えるが、意図しない副作用も生じてしまっている。

次のように、デコレータから返された値（上記で呼び出された関数）は、`fibonacci` という名前ではなくなってしまっている。

In [37]:
# Example 5
print(fibonacci)

<function trace.<locals>.wrapper at 0x00000169BFFB7F40>


これは、`trace` 関数が、その本体で定義する `wrapper` を返しており、デコレータの働きで `wrapper` 関数が、定義元のモジュールでの `fibonacci` という名前に代入されたのである。

この振る舞いは、デバッガ (「項目 80 `pdb` で対話的にデバッグすることを考える」参照)のようなイントロスペクションを行うツールの機能を損なうため、問題となってしまう。

例えば、デコレートされた `fibonacci` 関数には、組み込み関数 `help` が機能しない。

本来であれば、`help` を実行した際には、上で定義された `docstring` ('Return the n-th Fibonacci number') を出力すべきである。

In [38]:
# Example 6
help(fibonacci)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



オブジェクトシリアライザー (「項目 68 `copyreg` で `pickle` を信頼できるようにする」 参照) は、デコレートされた元の関数の位置を決定できないのでエラーとなってしまう。

In [39]:
import pickle
# Example 7
try:
    pickle.dumps(fibonacci)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_28564\2000568974.py", line 4, in <cell line: 3>
    pickle.dumps(fibonacci)
AttributeError: Can't pickle local object 'trace.<locals>.wrapper'


組み込みモジュール `functools` の `wraps` ヘルパー関数を使うことにより解決する。これは、デコレータを書くのを助けるデコレータである。これを `wrapper` 関数に適用すると、内部関数についてのすべての重要なメタデータが外部関数に複製される。

In [40]:
# Example 8
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

`help` 関数を実行すると、 関数をデコレートしたうえで、期待された結果が得られる。

In [41]:
# Example 9
help(fibonacci)

Help on function fibonacci in module __main__:

fibonacci(n)
    Return the n-th Fibonacci number



`pickle` オブジェクトシリアライザーも動作する。

In [42]:
# Example 10
print(pickle.dumps(fibonacci))


b'\x80\x04\x95\x1a\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\tfibonacci\x94\x93\x94.'


これらの例の他にも、Python 関数は、言語の関数インタフェースを保守するために保持しなくてはならない多くの標準属性 (例えば、`__name__`,`__module__`,`__annotations__`)を持っており、`wraps` を使うことにより、期待した動作を行うようになる。

### 覚えておくこと

- Python のデコレータ構文を使うと、 ある関数が他の関数を実行時に修正できる
- デコレータを使うことでデバッガのようなイントロスペクションを行うツールに奇妙な振る 舞いを引き起こすことがある
- 問題を起こさないようにデコレータを自分で定義するときには、組み込みモジュール functoolsのデコレータ wrapsを使う


# 第4章 内包表記とジェネレータ

多くのプログラム言語においてリスト、辞書、キー値対、集合を処理できるよう構築されている。Python は内包表記と呼ばれる特別な構文で、これらの型で簡潔にイテレーションしたり、派生データ構造を使えるようになっている。内包表記は、このようなよくある処理コードを読みやすくするだけでなく、 他にも多くの利点がある。

この処理スタイルは、 ジェネレータ関数にも拡張できて、関数により値のストリームを順に返すことができる。

ジェネレータ関数の呼び出しの結果は、 イテレータを使えるところであればどこでも使える（例: `for` ループ、アスタリスク付きの式）。ジェネレータは、性能を向上させ、メモリ使用を減らし、可読性を高める。

## 項目 27 map や filter の代わりにリスト内包表記を使う

Python は、他のシーケンスやイテラブルオブジェクトからリストを導出するための簡潔な構文を備えている。 この式は、リスト内包表記と呼ばれる。

例えば、リスト中の各数の平方を計算したいとする。次のコードは、これを簡単な `for` ループで実装している。

In [43]:
# Example 1
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
squares = []
for x in a:
    squares.append(x**2)
print(squares)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


これをリスト内包表記で実装すると次のようなコードになる。

ループする入力シーケンスに計算式を指定すれば、同じ結果が得られる。

In [44]:
# Example 2
squares = [x**2 for x in a]  # リスト内包表記
print(squares)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


単一引数関数でなければ、 単純な場合にはリスト内包表記のほうが組み込み関数 `map` よりも明確になる。

次のコードでは`map` を用いて実装したものであるが、計算するために `lambda` 関数を作る必要があり、見た目的にうるさく映ってしまっている。

In [45]:
# Example 3
alt = map(lambda x: x ** 2, a)
assert list(alt) == squares, f'{alt} {squares}'

`map` の場合と異なり、リスト内包表記では、入力リストから要素をフィルタリングし、対応する出力を結果から取り除くことが簡単にできる。 例えば、2で割り切れる数だけ平方を計算したいとき、リスト内包表記のループの後に条件式を付け加えるだけで実現できる。

In [46]:
# Example 4
even_squares = [x**2 for x in a if x % 2 == 0]
print(even_squares)

[4, 16, 36, 64, 100]


`map` を用いた実装でも、組み込み関数 `filter` を `map` と一緒に用いることで、同じ結果が得られるが、読みにくくなってしまう。


In [47]:
# Example 5
alt = map(lambda x: x**2, filter(lambda x: x % 2 == 0, a))
print(list(alt))

[4, 16, 36, 64, 100]


辞書と集合にもリスト内包表記に対応する表現式 (辞書内包表記と集合内包表記と呼ぶ)がある。 これらを用いて書く際にも、関連するデータ構造が容易に作成できる。

In [48]:
# Example 6
even_squares_dict = {x: x**2 for x in a if x % 2 == 0}
threes_cubed_set = {x**3 for x in a if x % 3 == 0}
print(even_squares_dict)
print(threes_cubed_set)

{2: 4, 4: 16, 6: 36, 8: 64, 10: 100}
{216, 729, 27}


`map` や `filter` でも対応するコンストラクタ呼び出しをラップすれば同じ結果を得ることができる。

これらの文は、長くなるため複数行に分割しなければならず、 さらに見た目がうるさくなるため、避けたほうが無難である。

In [62]:
# Example 7
alt_dict = dict(map(lambda x: (x, x**2),
				filter(lambda x: x % 2 == 0, a)))
alt_set = set(map(lambda x: x**3,
	          filter(lambda x: x % 3 == 0, a)))
print(even_squares_dict)
print(threes_cubed_set)

{2: 4, 4: 16, 6: 36, 8: 64, 10: 100}
{216, 729, 27}


### 覚えておくこと

- リスト内包表記は、`lambda` 式を必要としないので、組み込み関数の `map` や `filter` よりも明確
- リスト内包表記は、入力リストから要素を抜き出すのが容易なことに対し、`map` を用いた実装は `filter` が必要になり見た目がうるさくなってしまう
- 辞書と集合も内包表記を使って作成できる

## 項目28 内包表記では、3つ以上の式を避ける

基本的な使い方（「項目 27 `map` や `filter` の代わりにリスト内包表記を使う」 参照）の他に、内包表記では、多重ループもサポートしている。 例えば、行列（リストを要素とするリスト）を平坦化し、 1つのリストにすべての要素が含まれるようにする。これを、2つの `for` 式を含むリスト内包表記を使い、次のコードのように行う。式の実行順序は、左から右へとされている。

In [50]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flat = [x for row in matrix for x in row]
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


この例は単純で、読みやすく、 内包表記での多重ループの使い方も合理的である。 多重ループのもう1つの合理的な使い方は、次元2の入力リストを複製する場合である。例えば、2次元行列の各要素を二乗したいとする。次の内包表記は、前よりも `[]` が多くうるさいが、まだ読みやすい。

In [51]:
# Example 2
squared = [[x**2 for x in row] for row in matrix]
print(squared)

[[1, 4, 9], [16, 25, 36], [49, 64, 81]]


この内包表記にもう1つループを使うと、長くなりすぎてしまい、複数行に分割しないといけない。

In [52]:
# Example 3
my_lists = [
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]],
]
flat = [x for sublist1 in my_lists
        for sublist2 in sublist1
        for x in sublist2]
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


この時点で、複数行の内包表記は、他の書き方に比べてずっと短いという長所を失ってしまう。 

次の実装のように、通常の `for` ループ文を使って同じ結果が得られる。今回のケースではリスト内包表記で書くよりも、インデントによってループ構造がわかりやすくなっている。

In [53]:
# Example 4
flat = []
for sublist1 in my_lists:
    for sublist2 in sublist1:
        flat.extend(sublist2)
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


内包表記は、複数の `if` 条件を扱うこともできる。 同一のループでの複数条件は、明記しない場合 `and` 式となる。 例えば、数のリストから、4より大きな偶数だけをフィルターして取り出すコードを書くとする。次の2つのリスト内包表記は、いずれも同じ挙動を取る。

In [54]:
# Example 5
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = [x for x in a if x > 4 if x % 2 == 0]
c = [x for x in a if x > 4 and x % 2 == 0]
print(b)
print(c)
assert b and c
assert b == c

[6, 8, 10]
[6, 8, 10]


条件は、各レベルでの `for` 式の後に指定できます。 例えば、行列に対して、要素が3で割り切れて、 行方向での和が10以上というフィルターを考える。これは、リスト内包表記で簡潔に書くことができるが、読み解くのは困難である。

In [55]:
# Example 6
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
filtered = [[x for x in row if x % 3 == 0]
            for row in matrix if sum(row) >= 10]
print(filtered)

[[6], [9]]


この例は少しひねくれているが、実際には、このような式で十分な状況に出会うこともあるだろう。
このような `list`、`dict`、`set` の内包表記を使うことは避けた方が良い。このようなケースでこれらを使ってしまうと、読む側にとっては理解し難いものとなってしまう。`dict` 内包表記では、キーと値とで余分なバラメータを必要とするため、さらにひどいものとなってしまう。

 

大まかな基準を言うとすれば、リスト内包表記では、3つ以上の式を使うことは避けるべきである。2つの条件、2つのループ、1つの条件と1つのループまでとすることが好ましいだろう。

これよりも複雑になるのであれば、 通常の `if` 文や `for` 文を使った、ジェネレータのヘルパー関数を書くべきである(「項目30 リストを返さずにジェネレータを返すことを考える」参照)。


### 覚えておくこと

- 内包表記は多重ループと1つのループに複数の条件をサポートする
- 3つ以上の式を使う内包表記は、読むのが難しくなるため避けるべきである